# Introduction to XGBoost Spark3.0 with GPU

Taxi is an example of xgboost regressor. This notebook will show you how to load data, train the xgboost model and use this model to predict "fare_amount" of your taxi trip.

A few libraries required for this notebook:
  1. NumPy
  2. cudf jar
  3. xgboost4j jar
  4. xgboost4j-spark jar
  5. rapids-4-spark.jar  

This notebook also illustrates the ease of porting a sample CPU based Spark xgboost4j code into GPU. There is only one change required for running Spark XGBoost on GPU. That is replacing the API `setFeaturesCol(feature)` on CPU with the new API `setFeaturesCols(features)`. This also eliminates the need for vectorization (assembling multiple feature columns in to one column) since we can read multiple columns.

#### Import Required Libraries

In [1]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostRegressionModel, XGBoostRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time

Besides CPU version requires two extra libraries.
```Python
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
```

#### Create Spark Session and Data Reader

In [3]:
spark = SparkSession.builder.getOrCreate()
reader = spark.read

#### Specify the Data Schema and Load the Data

In [5]:
label = 'fare_amount'
schema = StructType([
    StructField('vendor_id', FloatType()),
    StructField('passenger_count', FloatType()),
    StructField('trip_distance', FloatType()),
    StructField('pickup_longitude', FloatType()),
    StructField('pickup_latitude', FloatType()),
    StructField('rate_code', FloatType()),
    StructField('store_and_fwd', FloatType()),
    StructField('dropoff_longitude', FloatType()),
    StructField('dropoff_latitude', FloatType()),
    StructField(label, FloatType()),
    StructField('hour', FloatType()),
    StructField('year', IntegerType()),
    StructField('month', IntegerType()),
    StructField('day', FloatType()),
    StructField('day_of_week', FloatType()),
    StructField('is_weekend', FloatType()),
])
features = [ x.name for x in schema if x.name != label ]

train_data = reader.schema(schema).option('header', True).csv('/data/taxi/csv/train')
trans_data  = reader.schema(schema).option('header', True).csv('/data/taxi/csv/test')

Note on CPU version, vectorization is required before fitting data to regressor, which means you need to assemble all feature columns into one column.

```Python
def vectorize(data_frame):
    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]
    return (VectorAssembler()
        .setInputCols(features)
        .setOutputCol('features')
        .transform(data_frame.select(to_floats))
        .select(col('features'), col(label)))

train_data = vectorize(train_data)
trans_data = vectorize(trans_data)
```

#### Create a XGBoostRegressor

In [6]:
params = { 
    'eta': 0.05,
    'treeMethod': 'gpu_hist',
    'maxDepth': 8,
    'subsample': 0.8,
    'gamma': 1.0,
    'numRound': 100,
    'numWorkers': 1,
}
regressor = XGBoostRegressor(**params).setLabelCol(label).setFeaturesCols(features)

The CPU version regressor provides the API `setFeaturesCol` which only accepts a single column name, so vectorization for multiple feature columns is required.
```Python
regressor = XGBoostRegressor(**params).setLabelCol(label).setFeaturesCol('features')
```

The parameter `num_workers` should be set to the number of GPUs in Spark cluster for GPU version, while for CPU version it is usually equal to the number of the CPU cores.

Concerning the tree method, GPU version only supports `gpu_hist` currently, while `hist` is designed and used here for CPU training.


#### Train the Data with Benchmark

In [7]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    print('{} takes {} seconds'.format(phrase, round(end - start, 2)))
    return result
model = with_benchmark('Training', lambda: regressor.fit(train_data))

Training takes 17.73 seconds


#### Save and Reload the Model

In [9]:
model.write().overwrite().save('/data/new-model-path')
loaded_model = XGBoostRegressionModel().load('/data/new-model-path')

#### Transformation and Show Result Sample

In [10]:
def transform():
    result = loaded_model.transform(trans_data).cache()
    result.foreachPartition(lambda _: None)
    return result
result = with_benchmark('Transformation', transform)
result.select('vendor_id', 'passenger_count', 'trip_distance', label, 'prediction').show(5)

Transformation takes 2.55 seconds
+------------+---------------+-------------+-----------+------------------+
|   vendor_id|passenger_count|trip_distance|fare_amount|        prediction|
+------------+---------------+-------------+-----------+------------------+
|1.55973043E9|            1.0|          1.1|        6.2| 5.670516490936279|
|1.55973043E9|            4.0|          2.7|        9.4|10.054250717163086|
|1.55973043E9|            1.0|          1.5|        6.1|  7.01417350769043|
|1.55973043E9|            1.0|          4.1|       12.6|14.309316635131836|
|1.55973043E9|            1.0|          4.6|       13.4|13.990922927856445|
+------------+---------------+-------------+-----------+------------------+
only showing top 5 rows



Note on CPU version: You cannot `select` the feature columns after vectorization. So please use `result.show(5)` instead.

#### Evaluation

In [11]:
accuracy = with_benchmark(
    'Evaluation',
    lambda: RegressionEvaluator().setLabelCol(label).evaluate(result))
print('RMSE is ' + str(accuracy))

Evaluation takes 0.45 seconds
RMSE is 3.3195416959403032


#### Stop

In [12]:
spark.stop()